for video

In [1]:
import cv2
import numpy as np
cap = cv2.VideoCapture("result_720.mp4")
fps = cap.get(cv2.CAP_PROP_FPS)
print(fps)
import matplotlib.pylab as plt


24.98819720779659


In [2]:
import cv2
import numpy as np
import time
 
class Profiler(object):
    def __enter__(self):
        self._startTime = time.time()
         
    def __exit__(self, type, value, traceback):
        print( "Elapsed time: {:.3f} sec".format(time.time() - self._startTime))

from timeit import default_timer as timer
def superimposition_mask(x,y):
    new = np.zeros(y.shape)
    for i in range(y.shape[2]):
        new[:,:,i] = x*y[:,:,i]
    return new

def mask(a, x1, x2, y1, y2, z1, z2):
    h_min = np.array((x1, y1, z1), np.uint8)
    h_max = np.array((x2, y2, z2), np.uint8)
    return cv2.inRange(a, h_min, h_max)

def im_cir(img,f):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV )
    thresh = mask(hsv, 0,238,166,255,157,255)
    new = superimposition_mask(thresh, img)
    #print(new[0][0])
    new = new.astype(np.uint8)
    #print(new[0][0])
    new = cv2.medianBlur(new,5)
    new = cv2.cvtColor(new, cv2.COLOR_BGR2GRAY)
    #cv2.imwrite('5.png',new)
    new = cv2.Canny(new, 350, 360)
    #cv2.imwrite('6.png',new)
    #hsv = cv2.cvtColor(new, cv2.COLOR_HSV2BGR )
    #hsv = cv2.cvtColor(hsv, cv2.COLOR_BGR2GRAY)
    circles = cv2.HoughCircles(new, cv2.HOUGH_GRADIENT, 1.15,0.0001,param1=85,param2=100,minRadius=5,maxRadius=300)
    print(circles.shape)
    try:
        circles = np.uint16(np.around(circles))
        for i in circles[0,:]:
    # нарисовать центры окружностей
            cv2.circle(img,(i[0],i[1]),i[2],(0,255,0),2)
            cv2.circle(img,(i[0],i[1]),2,(0,0,255),3)
    #cv2.imshow('result', new) 
        cv2.imwrite("videos/new"+str(f)+".jpg",img)
    except:
        print("xui")

In [4]:
cap = cv2.VideoCapture("result_720.mp4")
s=0
while True:
    s+=1
    with Profiler() as p:
        flag, img = cap.read()
        if flag:
            im_cir(img,s)
        print(flag)
        print(s)
   
    

(1, 1, 3)
True
1
Elapsed time: 0.113 sec
(1, 5, 3)
True
2
Elapsed time: 0.079 sec
(1, 6, 3)
True
3
Elapsed time: 0.074 sec
(1, 10, 3)
True
4
Elapsed time: 0.080 sec
(1, 4, 3)
True
5
Elapsed time: 0.080 sec
(1, 4, 3)
True
6
Elapsed time: 0.073 sec
(1, 7, 3)
True
7
Elapsed time: 0.083 sec
(1, 5, 3)
True
8
Elapsed time: 0.144 sec
(1, 4, 3)
True
9
Elapsed time: 0.123 sec
(1, 1, 3)
True
10
Elapsed time: 0.091 sec
(1, 1, 3)
True
11
Elapsed time: 0.071 sec
(1, 2, 3)
True
12
Elapsed time: 0.069 sec
(1, 4, 3)
True
13
Elapsed time: 0.077 sec
(1, 5, 3)
True
14
Elapsed time: 0.077 sec
(1, 5, 3)
True
15
Elapsed time: 0.081 sec
(1, 6, 3)
True
16
Elapsed time: 0.300 sec
(1, 5, 3)
True
17
Elapsed time: 0.099 sec
(1, 7, 3)
True
18
Elapsed time: 0.090 sec
(1, 11, 3)
True
19
Elapsed time: 0.076 sec
(1, 8, 3)
True
20
Elapsed time: 0.071 sec
(1, 10, 3)
True
21
Elapsed time: 0.076 sec
(1, 8, 3)
True
22
Elapsed time: 0.079 sec
(1, 8, 3)
True
23
Elapsed time: 0.104 sec
(1, 8, 3)
True
24
Elapsed time: 0.103 se

KeyboardInterrupt: 

In [ ]:
!mkdir videos


In [ ]:
!ls

In [ ]:
i =1 
cv2.imwrite("1"+str(2)+".png",img)